In [1]:
import os
os.chdir('../')

In [2]:
import re
import json
import time
import requests
from datetime import datetime

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from src.general_utils import util

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException

In [4]:
path_image_dir = '/Users/manish.sahu/Downloads/tiler/scrap-bookmyshow/data/wickedweasel'
path_image_list = util.list_list(path_image_dir, ('jpg', 'jpeg', 'png'))
n = len(path_image_list)
print(f'Numnber of images: {n}')

Numnber of images: 9241


In [42]:
# URL to scrape
url = "https://www.wickedweasel.com/en-us/outerwear/skirts"
base_product_url = 'https://www.wickedweasel.com'
path_output_dir = '/Users/manish.sahu/Downloads/tiler/scrap-bookmyshow/data/wickedweasel/images/shorts'
util.check_dir(path_output_dir)

# Send a GET request to the website
def parse_url(url:str):
    driver = webdriver.Firefox()
    try:
        # Set the page load timeout
        driver.set_page_load_timeout(10)

        # Open the URL
        print(f"Loading page: {url}")
        driver.get(url)
    except TimeoutException:
        print(f"Page load exceeded {10} seconds. Continuing with scraping...")


    # Try to close the popup if it appears
    try:
        # Locate the close button for the popup and click it
        popup_close_button = driver.find_element(By.CSS_SELECTOR, 'button.mfp-close')
        popup_close_button.click()
        time.sleep(2)  # Wait a moment for the popup to close
    except Exception as e:
        print("Popup not found or unable to close it:", e)

        
    # response = requests.get(url)
    # soup = BeautifulSoup(response.content, 'html.parser')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

In [43]:
soup = parse_url(url=url)

Loading page: https://www.wickedweasel.com/en-us/outerwear/skirts
Page load exceeded 10 seconds. Continuing with scraping...


In [57]:
# Find all the dress containers (adjust selectors based on the website's structure)
dress_containers = soup.find_all('div', class_='product-tile-container')

# Extract details for each dress
for dress in tqdm(dress_containers):
    try:
        product_name = dress.text.strip().replace(' ', '-')
        
        if product_name == 'promo':
            continue

        product_url = base_product_url + dress.find(class_='product-item')['href']
        path_image_dir = os.path.join(path_output_dir, product_name)
        util.check_dir(path_image_dir)

        product_soup = parse_url(url=product_url)
        
        image_list = product_soup.find_all(class_='carousel-container')[0].find_all('li')
        number_of_images = len(image_list)
        
        for i in range(number_of_images):
            image_url = image_list[i].find('img')['src']
            basename = os.path.basename(image_url).split('?')[0]
            path_image = os.path.join(path_image_dir, f'{basename}')

            if not os.path.exists(path_image):
                response = requests.get(image_url)
                with open(path_image, 'wb') as f:
                    f.write(response.content)
                # print(f"Downloaded: {path_image}")
            else:
                # print(f"Already exists: {path_image}")
                pass
    except Exception as e:
        print(f"Error: {e}")
        break
    
    # # title = dress.find('div', class_='product-name').text.strip()
    # # price = dress.find('span', class_='price').text.strip()
    # # link = dress.find('a', class_='product-link')['href']
    
    # print(f"Title: {title}")
    # print(f"Price: {price}")
    # print(f"Link: {link}")
    # print("-" * 40)



100%|██████████| 12/12 [02:49<00:00, 14.09s/it]


In [59]:
url = 'https://www.wickedweasel.com/en-us/bikinis/tri-top-bikini'


'bikinis'